In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.applications import imagenet_utils
from sklearn.metrics import confusion_matrix
import itertools
import os
import shutil
import random
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
os.chdir('PokemonData-with-greyscale')

In [3]:
folder_list = os.listdir()
folder_list

['train', 'validation']

In [4]:
if os.path.isdir('train/Abra/') is False: 
    os.mkdir('train')
    os.mkdir('validation')
    
    for folder in folder_list:
        shutil.move(f'{folder}', 'train')
        os.mkdir(f'validation/{folder}')
    
        validation_samples = random.sample(os.listdir(f'train/{folder}'), 10)
        for file_names in validation_samples:
            shutil.move(f'train/{folder}/{file_names}', f'validation/{folder}')
            
            
os.chdir('../..')            

In [5]:
# function to show images, taken from tensorflows website

def plotImages(images_arr):
    fig, axes = plt.subplots(1, 10, figsize=(20,20))
    axes = axes.flatten()
    for img, ax in zip( images_arr, axes):
        ax.imshow(img)
        ax.axis('off')
    plt.tight_layout()
    plt.show()

In [6]:
os.chdir('pokemon-classifier')

In [7]:
folder_list = os.listdir()
folder_list

['.DS_Store',
 '.git',
 '.gitattributes',
 '.ipynb_checkpoints',
 'app.py',
 'bw_generator.py',
 'Model_Training.ipynb',
 'PokemonData-with-greyscale',
 'pokemon_pred_mobilenet.h5',
 'templates']

In [8]:
os.chdir('PokemonData-with-greyscale')
folder_list = os.listdir()
folder_list

['train', 'validation']

In [9]:
os.chdir('train')
folder_list = os.listdir()
folder_list

['Abra',
 'Aerodactyl',
 'Alakazam',
 'Alolan Sandslash',
 'Arbok',
 'Arcanine',
 'Articuno',
 'Beedrill',
 'Bellsprout',
 'Blastoise',
 'Bulbasaur',
 'Butterfree',
 'Caterpie',
 'Chansey',
 'Charizard',
 'Charmander',
 'Charmeleon',
 'Clefable',
 'Clefairy',
 'Cloyster',
 'Cubone',
 'Dewgong',
 'Diglett',
 'Ditto',
 'Dodrio',
 'Doduo',
 'Dragonair',
 'Dragonite',
 'Dratini',
 'Drowzee',
 'Dugtrio',
 'Eevee',
 'Ekans',
 'Electabuzz',
 'Electrode',
 'Exeggcute',
 'Exeggutor',
 'Farfetchd',
 'Fearow',
 'Flareon',
 'Gastly',
 'Gengar',
 'Geodude',
 'Gloom',
 'Golbat',
 'Goldeen',
 'Golduck',
 'Golem',
 'Graveler',
 'Grimer',
 'Growlithe',
 'Gyarados',
 'Haunter',
 'Hitmonchan',
 'Hitmonlee',
 'Horsea',
 'Hypno',
 'Ivysaur',
 'Jigglypuff',
 'Jolteon',
 'Jynx',
 'Kabuto',
 'Kabutops',
 'Kadabra',
 'Kakuna',
 'Kangaskhan',
 'Kingler',
 'Koffing',
 'Krabby',
 'Lapras',
 'Lickitung',
 'Machamp',
 'Machoke',
 'Machop',
 'Magikarp',
 'Magmar',
 'Magnemite',
 'Magneton',
 'Mankey',
 'Marowak',
 '

In [10]:
pokemon_folders = os.listdir()
pokemon_folders

['Abra',
 'Aerodactyl',
 'Alakazam',
 'Alolan Sandslash',
 'Arbok',
 'Arcanine',
 'Articuno',
 'Beedrill',
 'Bellsprout',
 'Blastoise',
 'Bulbasaur',
 'Butterfree',
 'Caterpie',
 'Chansey',
 'Charizard',
 'Charmander',
 'Charmeleon',
 'Clefable',
 'Clefairy',
 'Cloyster',
 'Cubone',
 'Dewgong',
 'Diglett',
 'Ditto',
 'Dodrio',
 'Doduo',
 'Dragonair',
 'Dragonite',
 'Dratini',
 'Drowzee',
 'Dugtrio',
 'Eevee',
 'Ekans',
 'Electabuzz',
 'Electrode',
 'Exeggcute',
 'Exeggutor',
 'Farfetchd',
 'Fearow',
 'Flareon',
 'Gastly',
 'Gengar',
 'Geodude',
 'Gloom',
 'Golbat',
 'Goldeen',
 'Golduck',
 'Golem',
 'Graveler',
 'Grimer',
 'Growlithe',
 'Gyarados',
 'Haunter',
 'Hitmonchan',
 'Hitmonlee',
 'Horsea',
 'Hypno',
 'Ivysaur',
 'Jigglypuff',
 'Jolteon',
 'Jynx',
 'Kabuto',
 'Kabutops',
 'Kadabra',
 'Kakuna',
 'Kangaskhan',
 'Kingler',
 'Koffing',
 'Krabby',
 'Lapras',
 'Lickitung',
 'Machamp',
 'Machoke',
 'Machop',
 'Magikarp',
 'Magmar',
 'Magnemite',
 'Magneton',
 'Mankey',
 'Marowak',
 '

In [11]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img,img_to_array, load_img
datagen = ImageDataGenerator(rotation_range=40,
                            width_shift_range=0.2,
                            height_shift_range=0.2,
                            shear_range=0.2,
                            zoom_range=0.2,
                            horizontal_flip=True,
                            fill_mode='nearest',
                            channel_shift_range=100
                            )    


In [12]:
import re
augmented = False

images = os.listdir("Abra")
aug_images =  [i for i in images if re.search("^aug-image-", i)]

if len(aug_images) > 0:   
    augmented = True
    

In [13]:
if not augmented:
    for folder in pokemon_folders:
        os.chdir(folder)
        while(True):
            chosen_image = random.choice(os.listdir())
            if not chosen_image.startswith("gs") and chosen_image.endswith('.jpg'):
                break

        print(f'{folder}: {chosen_image}')
        current_dir = os.getcwd()
        img = load_img(chosen_image)
        x=img_to_array(img)
        x=x.reshape((1,) + x.shape)

        i=0
        for batch in datagen.flow(x, batch_size=1, save_to_dir=current_dir, 
                                  save_prefix='aug-image-', save_format='jpeg'):
            i +=1
            if i>=10:
                break


        os.chdir('../')       

    